# INFO 4271 - Exercise 3 - Indexing

Issued: April 30, 2024

Due: May 6, 2024

Please submit this filled sheet via Ilias by the due date.

---

# 1. Skip Pointers
Skip pointers can be used to accelerate posting list intersection, allowing pointers to be moved either to the next sequential list position or to the position of the skip pointer if one is available.

a) Implement the `intersect\_skip()` function sketched below. Each time you would ordinarily increment a pointer by one, you can alternatively follow a skip pointer, if one is available at the position.

In [9]:
#Intersect two sorted posting lists that contain skip pointers
def intersect_skip(A, B):
    res = []
    pointer_a, pointer_b = 0, 0
    
    while pointer_a < len(A) and pointer_b < len(B):
        
        # If the docID at the current index of A is equal to 
        # the docID at the current index of B, 
        # add it to the result and move both pointers forward
        if A[pointer_a][0] == B[pointer_b][0]:
            res.append(A[pointer_a][0])
            pointer_a += 1
            pointer_b += 1
        
        # cur docID in A < cur docID in B
        elif A[pointer_a][0] < B[pointer_b][0]:
            # No skip pointer or skip pointer too far
            if A[pointer_a][1] is None or A[pointer_a][2] > B[pointer_b][0]:
                pointer_a += 1
            # Move pointer to the skip pointer
            else:
                pointer_a = A[pointer_a][1]
        # cur docID in B < cur docID in A
        else:
            # Same thing
            if B[pointer_b][1] is None or B[pointer_b][2] > A[pointer_a][0]:
                pointer_b += 1
            else:
                pointer_b = B[pointer_b][1]

    return res

#Posting lists with skip pointers. 
#Entries take the form [docID, index to skip to, docID at that index]
times_skip = [[2, 3, 16], [12, None, None], [15, None, None], [16, 6, 27], [17, None, None], [23, None, None], [27, None, None]]
square_skip = [[3, 2, 12], [8, None, None], [12, 4, 23], [19, None, None], [23, None, None]]

print(intersect_skip(times_skip, square_skip))

[12, 23]


b) How many pointer increment operations did you need to intersect the two posting lists with the given skip pointers? How many operations would it have been for the same lists without skip pointers?

Using skip pointers, we reduce the number of operations by 1 as we can jump from document 3 to document 12 in B. Overall this results in 6 increment operations instead of 7. 

# 2. Positional Indices
Positional indices include for each posting the exact positions at which the term can be found in the document. This information allows us to satisfy two previously impossible types of queries. 1) Phrase queries require terms to occur adjacently to one another in a specific order. 2) Range queries allow for more leeway between term positions, merely requiring the two terms to appear within a specified number of tokens.

Implement the `intersect\_range()` function sketched in the code base. Each time you would ordinarily have reported a match, you will now need to check whether the range requirement is satisfied. As an optional addition, think about making this range check efficient using some of the techniques discussed for general posting list intersection.

In [10]:
#Intersect two sorted posting lists with document-internal proximity requirements.
def intersect_range(A, B, range):
    pointer_a, pointer_b = 0, 0
    res = []
    
    while pointer_a < len(A) and pointer_b < len(B):
        if A[pointer_a][0] == B[pointer_b][0]:
            # match -> check if positions are in range
            for pos_a in A[pointer_a][1]:
                for pos_b in B[pointer_b][1]:
                    if abs(pos_a - pos_b) <= range:
                        res.append(A[pointer_a][0])
                        break
            pointer_a += 1
            pointer_b += 1
        
        elif A[pointer_a][0] < B[pointer_b][0]:
            pointer_a += 1
        else:
            pointer_b += 1
            
    return res

def intersect_range_efficient(A, B, range):
    # Todo: later practice...
    pass

#Posting lists with document-internal positional information.
#Entries take the form [docID, [pos1, pos2, ...]]
times_range = [[2, [15, 128]], [12, [6, 45, 89, 942]], [15, [13]], [16, [1276, 1500]], [17, [13, 89, 90]], [23, [17, 64]], [27, [456, 629]]]
square_range = [[3, [65, 90]], [8, [67, 94]], [12, [3]], [19, [18, 81, 1881]], [23, [63]]]

print(intersect_range(times_range, square_range, 2))
print(intersect_range(times_range, square_range, 3))

[23]
[12, 23]


# 3. Paper Pick
Don't forget to submit your paper pick at https://forms.gle/JNnd1V7k1L3uPVxU6.